In [6]:
import pandas as pd
import seaborn as sns
import numpy as np

In [7]:
df = pd.read_csv('/content/drive/MyDrive/datasets/final_data_mice_protine.csv')

In [8]:
X = df.drop(['class'],axis=1)
y = df['class']


In [10]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
X[:] = scaler.fit_transform(X[:])

normalizer = preprocessing.Normalizer()
X[:] = normalizer.fit_transform(X[:])

min_max_scaler = preprocessing.MinMaxScaler()
X[:] = min_max_scaler.fit_transform(X[:])


In [5]:
X

,DYRK1A_N,ITSN1_N,BDNF_N,NR1_N,NR2A_N,pAKT_N,pBRAF_N,pCAMKII_N,pCREB_N,pELK_N,pERK_N,pJNK_N,PKCA_N,pMEK_N,pNR1_N,pNR2A_N,pNR2B_N,pPKCAB_N,pRSK_N,AKT_N,BRAF_N,CAMKII_N,CREB_N,ELK_N,ERK_N,GSK3B_N,JNK_N,MEK_N,TRKA_N,RSK_N,APP_N,Bcatenin_N,SOD1_N,MTOR_N,P38_N,pMTOR_N,DSCR1_N,AMPKA_N,NR2B_N,pNUMB_N,RAPTOR_N,TIAM1_N,pP70S6_N,NUMB_N,P70S6_N,pGSK3B_N,pPKCG_N,CDK5_N,S6_N,ADARB1_N,AcetylH3K9_N,RRP1_N,BAX_N,ARC_N,ERBB4_N,nNOS_N,Tau_N,GFAP_N,GluR3_N,GluR4_N,IL1B_N,P3525_N,pCASP9_N,PSD95_N,SNCA_N,Ubiquitin_N,pGSK3B_Tyr216_N,SHH_N,BAD_N,BCL2_N,pS6_N,pCFOS_N,SYP_N,H3AcK18_N,EGR1_N,H3MeK4_N,CaNA_N,Ts65Dn,Saline,S/C
0,0.324339,0.400112,0.877126,0.818550,0.923182,0.404223,0.438545,0.184623,0.590918,0.275198,0.333593,0.531677,0.761405,0.573644,0.792728,0.283651,0.791843,0.691358,0.521895,0.806929,0.267089,0.467487,0.392826,0.698082,0.842092,0.807674,0.714483,0.668156,0.773919,0.384554,0.497512,0.856440,0.144609,0.511331,0.205444,0.622164,0.527372,0.773976,0.481786,0.549345,0.462335,0.593580,0.260984,0.447050,0.369554,0.731995,0.368875,0.250803,0.300503,0.428975,0.181046,0.381275,0.633114,0.310213,0.332100,0.500491,0.106263,0.331312,0.495031,0.268459,0.220472,0.188460,0.453646,0.506777,0.086762,0.257511,0.386124,0.217306,0.173889,0.325841,0.310213,0.239247,0.444584,0.103264,0.130201,0.072320,0.652553,0.196058,0.195952,0.214656
1,0.342015,0.363496,0.857200,0.841665,0.906199,0.380656,0.403352,0.148150,0.571780,0.244671,0.348341,0.503125,0.727629,0.521577,0.705837,0.255309,0.714791,0.608659,0.530907,0.799337,0.255489,0.415486,0.349596,0.680077,0.816581,0.817693,0.682598,0.615458,0.740336,0.323649,0.452800,0.846764,0.115186,0.409616,0.167107,0.552906,0.475625,0.710680,0.418122,0.477462,0.398835,0.530530,0.231611,0.447459,0.361416,0.783208,0.358121,0.249545,0.287188,0.417973,0.177053,0.376761,0.600541,0.290825,0.383872,0.508937,0.105049,0.353689,0.545969,0.272732,0.257898,0.220877,0.513974,0.489273,0.051167,0.194091,0.413620,0.252499,0.109538,0.322969,0.290825,0.191684,0.477667,0.076919,0.119819,0.048290,0.717586,0.166012,0.165902,0.181760
2,0.334743,0.395036,0.868378,0.756004,0.875591,0.375672,0.424229,0.153500,0.588728,0.234567,0.334298,0.478287,0.695208,0.522288,0.788281,0.272203,0.715258,0.589801,0.586927,0.762067,0.254301,0.463513,0.353047,0.670879,0.836673,0.803036,0.699508,0.647430,0.743255,0.350499,0.431506,0.844965,0.120075,0.416484,0.172812,0.547201,0.475373,0.650662,0.431706,0.462648,0.361544,0.506041,0.215946,0.463787,0.376068,0.825140,0.387713,0.273940,0.339597,0.398972,0.192622,0.354676,0.657258,0.320196,0.324909,0.494423,0.107308,0.350073,0.574555,0.272355,0.388158,0.213751,0.504389,0.498911,0.051890,0.188117,0.409275,0.225313,0.129568,0.323788,0.320196,0.211268,0.463018,0.082010,0.118827,0.042432,0.818711,0.174579,0.174471,0.191140
3,0.279753,0.303111,0.707963,0.626022,0.783441,0.405324,0.420670,0.076231,0.440135,0.252889,0.281320,0.483188,0.741106,0.680682,0.563379,0.158019,0.610575,0.706425,0.632812,0.650567,0.242333,0.442916,0.393822,0.390004,0.653736,0.785708,0.720207,0.489927,0.676701,0.349399,0.403846,0.641187,0.091601,0.412620,0.143970,0.531744,0.466147,0.606079,0.416920,0.478921,0.437317,0.509506,0.356563,0.267769,0.200952,0.728722,0.408413,0.259631,0.141446,0.348467,0.153266,0.396396,0.620310,0.193388,0.224091,0.373936,0.020188,0.339386,0.543528,0.299256,0.148118,0.130676,0.352254,0.437535,0.069412,0.116547,0.380406,0.179705,0.183991,0.317820,0.193388,0.214033,0.315227,0.110192,0.145681,0.069300,0.735659,0.112141,0.112024,0.122779
4,0.272367,0.303483,0.714082,0.537505,0.712531,0.371640,0.395467,0.062402,0.323633,0.225661,0.256533,0.438156,0.674238,0.511103,0.541529,0.136657,0.539644,0.649630,0.654983,0.578154,0.220364,0.404039,0.347956,0.423633,0.625936,0.734812,0.668112,0.455171,0.613122,0.285226,0.366218,0.618055,0.074327,0.334495,0.096031,0.446855,0.444210,0.497233,0.344497,0.419274,0.353793,0.425501,0.350319,0.247366,0.203811,0.754526,0.420675,0.261963,0.169200,0.355119,0.157075,0.379567,0.659212,0.210296,0.236310,0.404858

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=100)

In [12]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
classifier.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [13]:
y_pred = classifier.predict(X_test)

In [14]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
cm = confusion_matrix(y_test,y_pred)
score = accuracy_score(y_test,y_pred)
print(cm)
print(score)

[[43  0  0  0  0  0  0  0]
 [ 0 40  0  0  0  0  0  0]
 [ 0  0 59  0  0  0  0  0]
 [ 0  0  0 41  0  0  0  0]
 [ 0  0  0  0 36  0  0  0]
 [ 0  0  0  0  0 27  0  0]
 [ 0  0  0  0  0  0 31  0]
 [ 0  0  0  0  0  0  0 47]]
1.0


In [15]:
from sklearn.model_selection import RandomizedSearchCV

In [16]:
#Randomized Search CV

# Number of trees in Xgboost
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 200, num = 8)]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(1, 10, num = 3)]
min_samples_split = [2, 5, 10, 15]
min_samples_leaf = [1, 2, 5] 

In [17]:
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_leaf':min_samples_leaf,
              'min_samples_split':min_samples_split}

print(random_grid)

{'n_estimators': [10, 37, 64, 91, 118, 145, 172, 200], 'max_depth': [1, 5, 10], 'min_samples_leaf': [1, 2, 5], 'min_samples_split': [2, 5, 10, 15]}


In [18]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
classifier=RandomForestClassifier()

In [ ]:
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations
rf_random = RandomizedSearchCV(estimator = classifier, param_distributions = random_grid,scoring='accuracy', n_iter =51, cv = 5, verbose=2, random_state=42, n_jobs = -1)

In [ ]:
rf_random.fit(X_train,y_train)

Fitting 5 folds for each of 51 candidates, totalling 255 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:   29.0s
[Parallel(n_jobs=-1)]: Done 255 out of 255 | elapsed:   54.8s finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [ ]:
rf_random.best_params_


{'max_depth': 5,
 'min_samples_leaf': 2,
 'min_samples_split': 15,
 'n_estimators': 145}

In [ ]:
y_pred_ht = rf_random.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
cm=confusion_matrix(y_test,y_pred)
score=accuracy_score(y_test,y_pred)
print(cm)
print(score)

[[43  0  0  0  0  0  0  0]
 [ 0 40  0  0  0  0  0  0]
 [ 0  0 59  0  0  0  0  0]
 [ 0  0  0 41  0  0  0  0]
 [ 0  0  0  0 36  0  0  0]
 [ 0  0  0  0  0 27  0  0]
 [ 0  0  0  0  0  0 31  0]
 [ 0  0  0  0  0  0  0 47]]
1.0
